In [1]:
import requests
#import json
from pyspark.sql import SparkSession, SQLContext, Row 
from pyspark.sql.types import *
from pyspark import SparkContext
spark = SparkSession.builder.master("local").appName("north_labs").getOrCreate();


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/08 11:34:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/08 11:34:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
ceps = ['03171060','11065201', '03667000', '00011']
logradouro = []
cidade = []

##file = open('/tmp/test_northlabs.csv','a')
### função para chamada da API
def consulta_cep():
    for cep in ceps:
        if len(cep) == 8:
            request = requests.get('https://viacep.com.br/ws/{}/json/'.format(cep))
            cep_data = request.json()
            #print(cep_data)         
            #print(request.json())
            #print('cidade {}'.format(cep_data['localidade']))
            logradouro.append(cep_data['logradouro'] + "," + cep_data['localidade']) 
            cidade.append(cep_data['localidade'])

            #dump         = [json.dumps(cep_data)]
            #jsonRDD      = sc.parallelize(dump)
            #resultado    = spark.read.json(jsonRDD)
            #resultado.show()

        else: 
            print(f'cep errado ! {cep}')

    return cep_data

df_json = consulta_cep()


### gera o dataframe baseado na lista gerada durante a leitura do json/retorno da API
df_cidade = spark.createDataFrame(cidade, StringType())

### gera o data frame baseado no json recebido da API
df = spark.createDataFrame([df_json])

### cria uma tabela/view a partir do data frame
df.createOrReplaceTempView('API')

### executa query na tabela/view gerada 
df_select = spark.sql('select * from API')
df_select.show()

#grava csv porem em formato de subpasta com nome incorreto 
df_select.coalesce(1).write.format('csv').mode('overwrite').option("header", "true").option("delimiter",",").save('/tmp/test_northlabs')


df_read = spark.read.format('csv').option('header', True).load('/tmp/test_northlabs')
df_read.show()


### ordena cidade e depois mostra somente top 3
#select cidade, qtd from ( select cidade, count(*) as qtd from mawe_silver.clientes group by cidade )  order by 2 desc limit 3
#ou 
#select * from (
#select cidade, qtd, rank() over(order by qtd desc ) as ordenacao from (
#select cidade, count(*) as qtd from mawe_silver.clientes group by cidade ) ) where ordenacao < 4




#with open('/tmp/prime.txt', 'a') as prime_file:
#f.write("\nNew Line")




#df_select.write.csv("/tmp/df_select.csv")
#df_select.write.parquet("/tmp/df_select.parquet")

#df_select.write.format('csv').save('/tmp/north.csv')

#df_withcol = data_rdd.toDF(['height','color','width'])

#df_withcol = df_json.toDF(['coluna1','coluna2','coluna3','coluna4','coluna5','coluna6','coluna7','coluna8','coluna9','coluna10'])

#df_cidade.write.csv("/tmp/df_cidade.csv")
 

##df_resultado = spark.read.json(cidade)

#print(df_json)

#df_leitura_jason = spark.read.json('/tmp/prime.txt')










cep errado ! 00011
+-------+---------+-----------+---+----+-------+----------+-----------------+-----+---+
| bairro|      cep|complemento|ddd| gia|   ibge|localidade|       logradouro|siafi| uf|
+-------+---------+-----------+---+----+-------+----------+-----------------+-----+---+
|Vila Ré|03667-000|           | 11|1004|3550308| São Paulo|Rua Baltazar Brum| 7107| SP|
+-------+---------+-----------+---+----+-------+----------+-----------------+-----+---+

+-------+---------+-----------+---+----+-------+----------+-----------------+-----+---+
| bairro|      cep|complemento|ddd| gia|   ibge|localidade|       logradouro|siafi| uf|
+-------+---------+-----------+---+----+-------+----------+-----------------+-----+---+
|Vila Ré|03667-000|       null| 11|1004|3550308| São Paulo|Rua Baltazar Brum| 7107| SP|
+-------+---------+-----------+---+----+-------+----------+-----------------+-----+---+



In [6]:
df_select = spark.sql('select bairro from API')
df_select.show()

+-------+
| bairro|
+-------+
|Vila Ré|
+-------+



In [ ]:
#from pyspark.sql import SparkSession
#from pyspark.sql.functions import *
#spark = SparkSession.builder.master("local").appName("north_labs").getOrCreate();

In [ ]:
##df = spark.read.parquet('C:\Saulo\Ambev\Arquivos Parquet\ds_zz_Product')

In [ ]:
#df_parquet = spark.sql('''create database if not exists db_northlabs ''')
#df_parquet = spark.sql('''create table if not exists db_northlabs.InvoiceParquet using parquet location "C:\Saulo\Ambev\ds_zz_Product"''')
#df_parquet = spark.sql('''select distinct InvoiceNumber from db_parquet.InvoiceParquet  where InvoiceNumber <> 0536884625''')
#df_parquet.show()